In [24]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
import re
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Importing Data

In [25]:
dataset_fake = pd.read_csv("sample_data/Fake.csv")
dataset_fake["label"]=1
dataset_true = pd.read_csv("sample_data/True.csv")
dataset_true["label"]=0

# Making a single dataset with random inputs
dataset = pd.concat((dataset_fake,dataset_true), axis=0)
dataset = dataset.sample(frac=1).reset_index(drop=True)

X = dataset.title
y = dataset.label

Data Preprocessing

In [26]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

corpus = []
for i in range(len(X)):
    news = re.sub( "[^a-zA-Z]"," ",X[i])
    news = news.lower()
    news = news.split()
    sub_corpus = [ps.stem(word) for word in news if not word in stopwords.words("english")]
    sub_corpus = " ".join(sub_corpus)
    corpus.append(sub_corpus)

Model Creation

In [27]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot

In [28]:
input_len = 1000
oh = [one_hot(word, input_len) for word in corpus]

In [29]:
ohp = pad_sequences(oh, padding = "post")
sent_len = len(ohp[0])

In [30]:
output_len = 50
model = Sequential()
model.add(Embedding(input_len, output_len, input_length=sent_len))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 35, 50)            50000     
_________________________________________________________________
dropout_2 (Dropout)          (None, 35, 50)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 110,501
Trainable params: 110,501
Non-trainable params: 0
_________________________________________________________________
None


Data Training

In [31]:
X_final = np.array(ohp)
y_final = np.array(y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size =0.2, random_state=20)

In [32]:
model.fit(X_train,y_train,validation_data=(X_test,y_test), epochs=10, batch_size=64)
y_pred = model.predict_classes(X_test)

Epoch 1/10
562/562 [==============================] - 4s 8ms/step - loss: 0.3255 - accuracy: 0.8487 - val_loss: 0.2684 - val_accuracy: 0.8928
Epoch 2/10
562/562 [==============================] - 4s 7ms/step - loss: 0.2468 - accuracy: 0.8991 - val_loss: 0.2468 - val_accuracy: 0.8979
Epoch 3/10
562/562 [==============================] - 4s 7ms/step - loss: 0.2256 - accuracy: 0.9056 - val_loss: 0.2279 - val_accuracy: 0.9056
Epoch 4/10
562/562 [==============================] - 4s 7ms/step - loss: 0.2097 - accuracy: 0.9126 - val_loss: 0.2409 - val_accuracy: 0.8899
Epoch 5/10
562/562 [==============================] - 4s 7ms/step - loss: 0.2031 - accuracy: 0.9144 - val_loss: 0.2253 - val_accuracy: 0.9024
Epoch 6/10
562/562 [==============================] - 4s 7ms/step - loss: 0.1915 - accuracy: 0.9182 - val_loss: 0.2191 - val_accuracy: 0.9156
Epoch 7/10
562/562 [==============================] - 4s 7ms/step - loss: 0.1805 - accuracy: 0.9237 - val_loss: 0.2041 - val_accuracy: 0.9198
Epoch 

Model Analysis

In [33]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3985,  232],
       [ 517, 4246]])

In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9165924276169265